In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Bidirectional,Dense, LSTM, Activation, Dropout, Flatten
from keras.layers import LeakyReLU
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from keras.models import Sequential
import tensorflow as tf
from keras.regularizers import l2
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report,roc_curve,auc

In [2]:
test = pd.read_excel("447_feature_test.xlsx")

# NetSurfP2.0

In [15]:
X_test_Net = test.iloc[:,1783:1983]
y_test_Net = test["Target"]


print(X_test_Net.isnull().sum().sum())
X_test_Net.fillna(0,inplace=True)
print(X_test_Net.isnull().sum().sum())

scaler = MinMaxScaler()


X_test_Net = scaler.fit_transform(X_test_Net)


X_test_Net = np.array(X_test_Net)
y_test_Net = np.array(y_test_Net)

664
0


# Gapped Dipeptide Features

In [14]:
X_test_GD = test.iloc[:,1983:2007]
y_test_GD = test["Target"]


scaler = MinMaxScaler()


X_test_GD = scaler.fit_transform(X_test_GD)


X_test_GD = np.array(X_test_GD)
y_test_GD = np.array(y_test_GD)

# PSSM Features

In [13]:
X_test_PSSM = test.iloc[:,1283:1783]
y_test_PSSM = test["Target"]


print(X_test_PSSM.isnull().sum().sum())
X_test_PSSM.fillna(0,inplace=True)
print(X_test_PSSM.isnull().sum().sum())

scaler = MinMaxScaler()


X_test_PSSM = scaler.fit_transform(X_test_PSSM)


X_test_PSSM = np.array(X_test_PSSM)
y_test_PSSM = np.array(y_test_PSSM)

1660
0


# Combine Three Features (NetSurfP-2.0, Gapped Dipeptide, PSSM) and Check the size of the matrix, shuffle it

In [16]:
X_test_Net_PSSM_GD = np.hstack((X_test_Net,X_test_PSSM,X_test_GD))

y_test_Net_PSSM_GD = y_test_Net


X_test, y_test = shuffle(X_test_Net_PSSM_GD, y_test_Net_PSSM_GD, random_state=7)
X_test = np.array(X_test)
y_test = np.array(y_test)


print(X_test.shape,y_test.shape)

(447, 724) (447,)


# Load the model and see the performance matrices

In [29]:
model = keras.models.load_model("3080_NetSurfP2.0_PSSM_GD"+str(8)+".h5")
Y_pred = model.predict(X_test)
Y_pred = (Y_pred > 0.5)
y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
y_pred = np.array(y_pred)
print("Matthews Correlation : ",matthews_corrcoef(y_test, y_pred))
print("Confusion Matrix : \n",confusion_matrix(y_test, y_pred))
print("Accuracy on test set:   ",accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
print(mcc)

TP = cm[1][1]
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]

Sensitivity = TP/(TP+FN)

Specificity = TN/(TN+FP)

print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)

print(classification_report(y_test, y_pred))

fpr, tpr, _ = roc_curve(y_test, y_pred)

roc_auc_test = auc(fpr,tpr)

print("Area Under Curve:   ",roc_auc_test)

Matthews Correlation :  0.5310682030023739
Confusion Matrix : 
 [[227  53]
 [ 46 121]]
Accuracy on test set:    0.7785234899328859
0.5310682030023739
Sensitivity:    0.7245508982035929 	 Specificity:    0.8107142857142857
              precision    recall  f1-score   support

           0       0.83      0.81      0.82       280
           1       0.70      0.72      0.71       167

    accuracy                           0.78       447
   macro avg       0.76      0.77      0.77       447
weighted avg       0.78      0.78      0.78       447

Area Under Curve:    0.7676325919589393
